In [1]:
import sqlite3
import smtplib
from email.mime.text import MIMEText
from datetime import datetime, timedelta

In [2]:
# Koneksi ke database SQLite
def create_connection():
    conn = sqlite3.connect('invoices.db')
    return conn

In [3]:
# Membuat tabel invoices
def create_table():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS invoices (
            id INTEGER PRIMARY KEY,
            amount REAL NOT NULL,
            due_date TEXT NOT NULL,
            recipient TEXT NOT NULL,
            status TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

In [4]:
# Menambahkan faktur
def add_invoice(amount, due_date, recipient):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO invoices (amount, due_date, recipient, status)
        VALUES (?, ?, ?, ?)
    ''', (amount, due_date, recipient, 'Belum Dibayar'))
    conn.commit()
    conn.close()

In [5]:
# Mengirim email pengingat
def send_reminder(recipient, amount, due_date):
    sender_email = "your_email@example.com"  # Ganti dengan email Anda
    sender_password = "your_password"  # Ganti dengan password email Anda

    msg = MIMEText(f"Pengingat: Anda memiliki faktur sebesar {amount} yang jatuh tempo pada {due_date}.")
    msg['Subject'] = 'Pengingat Pembayaran Faktur'
    msg['From'] = sender_email
    msg['To'] = recipient

    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)

In [6]:
# Melacak dan mengirim pengingat untuk faktur yang jatuh tempo
def check_due_invoices():
    conn = create_connection()
    cursor = conn.cursor()
    today = datetime.now().date()
    cursor.execute('''
        SELECT * FROM invoices WHERE status = 'Belum Dibayar' AND due_date <= ?
    ''', (today,))
    invoices = cursor.fetchall()
    for invoice in invoices:
        id, amount, due_date, recipient, status = invoice
        send_reminder(recipient, amount, due_date)
        print(f"Pengingat dikirim ke {recipient} untuk faktur {id}.")
    conn.close()

In [7]:
# Menandai faktur sebagai sudah dibayar
def mark_as_paid(invoice_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('''
        UPDATE invoices SET status = 'Sudah Dibayar' WHERE id = ?
    ''', (invoice_id,))
    conn.commit()
    conn.close()

# Fungsi utama
def main():
    create_table()
    while True:
        print("\nMenu:")
        print("1. Tambah Faktur")
        print("2. Kirim Pengingat")
        print("3. Tandai Faktur Sudah Dibayar")
        print("4. Keluar")
        choice = input("Pilih opsi: ")

        if choice == '1':
            amount = float(input("Masukkan jumlah faktur: "))
            due_date = input("Masukkan tanggal jatuh tempo (YYYY-MM-DD): ")
            recipient = input("Masukkan email penerima: ")
            add_invoice(amount, due_date, recipient)
            print("Faktur berhasil ditambahkan.")

        elif choice == '2':
            check_due_invoices()

        elif choice == '3':
            invoice_id = int(input("Masukkan ID faktur yang sudah dibayar: "))
            mark_as_paid(invoice_id)
            print("Faktur berhasil ditandai sebagai sudah dibayar.")

        elif choice == '4':
            break

        else:
            print("Opsi tidak valid. Silakan coba lagi.")

if __name__ == "__main__":
    main()


Menu:
1. Tambah Faktur
2. Kirim Pengingat
3. Tandai Faktur Sudah Dibayar
4. Keluar
Pilih opsi: 1
Masukkan jumlah faktur: 250000
Masukkan tanggal jatuh tempo (YYYY-MM-DD): 2024-12-20
Masukkan email penerima: diradhn1611@gmail.com
Faktur berhasil ditambahkan.

Menu:
1. Tambah Faktur
2. Kirim Pengingat
3. Tandai Faktur Sudah Dibayar
4. Keluar
Pilih opsi: 4


In [8]:
# Koneksi ke database
conn = sqlite3.connect('invoices.db')
cursor = conn.cursor()

# Memeriksa tabel yang ada
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tabel yang ada:", tables)

# Menampilkan data dari tabel invoices
cursor.execute("SELECT * FROM invoices;")
rows = cursor.fetchall()
print("Data dalam tabel invoices:")
for row in rows:
    print(row)

# Menutup koneksi
conn.close()

Tabel yang ada: [('invoices',)]
Data dalam tabel invoices:
(1, 250000.0, '2024-12-20', 'diradhn1611@gmail.com', 'Belum Dibayar')


In [9]:
def delete_all_invoices():
    conn = sqlite3.connect('invoices.db')
    cursor = conn.cursor()
    cursor.execute("DELETE FROM invoices")
    conn.commit()
    conn.close()
    print("Semua data dalam tabel invoices berhasil dihapus.")

delete_all_invoices()

Semua data dalam tabel invoices berhasil dihapus.
